### Developing a real stratergy to invest in the stock market

When developing our stratergy, the goal will be to acquire good and useful information about each stock. We then need to check which parameters play a role in explaining variation in the data and which simply contribute noise to the system. Once we have generated and calculated the financial indicators and then optimised the tuning of these params using something like simulated annealing to search the complex solution space we can test and compare our stratergy against the S&P500 real life data. 

Sentiment data could be useful, financial statements even better, unsure how to extract, useful information from these datasets.

Looking for monthly readjustments for our stock portfolio. Further investigation should be carried 
on arbitrage. 

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import os

In [2]:
tickers = ["AAPL", "MSFT", "GOOGL"]
start_date = "2015-01-01"
end_date = "2023-12-31"

In [4]:
# def download_stock_data(tickers, start_date, end_date):
#     stock_data = yf.download(tickers, start_date, end_date)
#     return stock_data

data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker')
data

# data = download_stock_data(ticker, start_date, end_date)

# data_folder = "C:/Users/codyr/Desktop/Coding_/Quant_Trading_Basics"

data = pd.read_csv("C:/Users/codyr/Desktop/Coding_/Quant_Trading_Basics/AAPL_stock_data.csv")

data.head()

# data.to_csv('AAPL_stock_data.csv', index=True)

[*********************100%***********************]  3 of 3 completed


Ticker            MSFT                                                  \
Price             Open        High         Low       Close   Adj Close   
Date                                                                     
2015-01-02   46.660000   47.419998   46.540001   46.759998   40.232845   
2015-01-05   46.369999   46.730000   46.250000   46.330002   39.862869   
2015-01-06   46.380001   46.750000   45.540001   45.650002   39.277802   
2015-01-07   45.980000   46.459999   45.490002   46.230000   39.776836   
2015-01-08   46.750000   47.750000   46.720001   47.590000   40.946995   
...                ...         ...         ...         ...         ...   
2023-12-22  373.679993  375.179993  372.709991  374.579987  372.543945   
2023-12-26  375.000000  376.940002  373.500000  374.660004  372.623505   
2023-12-27  373.690002  375.059998  372.809998  374.070007  372.036713   
2023-12-28  375.369995  376.459991  374.160004  375.279999  373.240112   
2023-12-29  376.000000  377.160004  373.480011  376.040009  373.995972   

Ticker                     GOOGL                                      \
Price         Volume        Open        High         Low       Close   
Date                                                                   
2015-01-02  27913900   26.629999   26.790001   26.393999   26.477501   
2015-01-05  39673900   26.357500   26.399500   25.887501   25.973000   
2015-01-06  36447900   26.025000   26.060499   25.277500   25.332001   
2015-01-07  29114100   25.547501   25.574499   25.182501   25.257500   
2015-01-08  29645200   25.075500   25.375000   24.750999   25.345501   
...              ...         ...         ...         ...         ...   
2023-12-22  17091100  140.770004  141.990005  140.710007  141.490005   
2023-12-26  12673100  141.589996  142.679993  141.190002  141.520004   
2023-12-27  14905400  141.589996  142.080002  139.889999  140.369995   
2023-12-28  14327000  140.779999  141.139999  139.750000  140.229996   
2023-12-29  18723000  139.630005  140.360001  138.779999  139.690002   

Ticker                                  AAPL                          \
Price        Adj Close    Volume        Open        High         Low   
Date                                                                   
2015-01-02   26.381865  26480000   27.847500   27.860001   26.837500   
2015-01-05   25.879185  41182000   27.072500   27.162500   26.352501   
2015-01-06   25.240503  54456000   26.635000   26.857500   26.157499   
2015-01-07   25.166271  46918000   26.799999   27.049999   26.674999   
2015-01-08   25.253954  73054000   27.307501   28.037500   27.174999   
...                ...       ...         ...         ...         ...   
2023-12-22  140.978943  26514600  195.179993  195.410004  192.970001   
2023-12-26  141.008835  16780300  193.610001  193.889999  192.830002   
2023-12-27  139.862976  19628600  192.490005  193.500000  191.089996   
2023-12-28  139.723480  16045700  194.139999  194.660004  193.169998   
2023-12-29  139.185440  18727200  193.899994  194.399994  191.729996   

Ticker                                         
Price            Close   Adj Close     Volume  
Date                                           
2015-01-02   27.332500   24.347176  212818400  
2015-01-05   26.562500   23.661270  257142000  
2015-01-06   26.565001   23.663500  263188400  
2015-01-07   26.937500   23.995316  160423600  
2015-01-08   27.972500   24.917271  237458000  
...                ...         ...        ...  
2023-12-22  193.600006  192.656189   37122800  
2023-12-26  193.050003  192.108856   28919300  
2023-12-27  193.149994  192.208359   48087700  
2023-12-28  193.580002  192.636261   34049900  
2023-12-29  192.529999  191.591400   42628800  

[2264 rows x 18 columns]

In [16]:
multi_index_data = (
    data.stack(level=0, future_stack=True)
    .swaplevel()
    .sort_index()
)

multi_index_data

Price                    Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
AAPL   2015-01-02   27.847500   27.860001   26.837500   27.332500   24.347176   
       2015-01-05   27.072500   27.162500   26.352501   26.562500   23.661270   
       2015-01-06   26.635000   26.857500   26.157499   26.565001   23.663500   
       2015-01-07   26.799999   27.049999   26.674999   26.937500   23.995316   
       2015-01-08   27.307501   28.037500   27.174999   27.972500   24.917271   
...                       ...         ...         ...         ...         ...   
MSFT   2023-12-22  373.679993  375.179993  372.709991  374.579987  372.543945   
       2023-12-26  375.000000  376.940002  373.500000  374.660004  372.623505   
       2023-12-27  373.690002  375.059998  372.809998  374.070007  372.036713   
       2023-12-28  375.369995  376.459991  374.160004  375.279999  373.240112   
       2023-12-29  376.000000  377.160004  373.480011  376.040009  373.995972   

Price                 Volume  
Ticker Date                   
AAPL   2015-01-02  212818400  
       2015-01-05  257142000  
       2015-01-06  263188400  
       2015-01-07  160423600  
       2015-01-08  237458000  
...                      ...  
MSFT   2023-12-22   17091100  
       2023-12-26   12673100  
       2023-12-27   14905400  
       2023-12-28   14327000  
       2023-12-29   18723000  

[6792 rows x 6 columns]

In [46]:
# data = data.iloc[2:]

#data.rename(columns={data.columns[0]: 'Date'}, inplace=True)

# data['Date'] = pd.to_datetime(data['Date'])
# data.set_index('Date', inplace=True)

# data

,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2020-01-02,72.79600524902344,75.0875015258789,75.1500015258789,73.79750061035156,74.05999755859375,135480400
2020-01-03,72.08830261230469,74.35749816894531,75.1449966430664,74.125,74.2874984741211,146322800
2020-01-06,72.6627197265625,74.94999694824219,74.98999786376953,73.1875,73.44750213623047,118387200
2020-01-07,72.32095336914062,74.59750366210938,75.2249984741211,74.37000274658203,74.95999908447266,108872000
2020-01-08,73.48432159423828,75.79750061035156,76.11000061035156,74.29000091552734,74.29000091552734,132079200
...,...,...,...,...,...,...
2023-12-05,192.47705078125,193.4199981689453,194.39999389648438,190.17999267578125,190.2100067138672,66628400
2023-12-06,191.38241577148438,192.32000732421875,194.75999450683594,192.11000061035156,194.4499969482422,41089700
2023-12-07,193.32290649414062,194.27000427246094,195.0,193.58999633789062,193.6300048828125,47477700


In [2]:
### Define and calcuate RSI 
def calculate_rsi(data, column, window=14):

    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input data must be a pandas DataFrame")

    if column not in data.columns:
        raise ValueError(f"Column '{column}' not found in the DataFrame")

    # Calculate daily price changes (delta)
    delta = data[column].diff()

    # Separate gains and losses
    gains = delta.clip(lower=0)
    losses = -delta.clip(upper=0)
    
    avg_gain = gains.rolling(window=window, min_periods=window).mean()
    avg_loss = losses.rolling(window=window, min_periods=window).mean()

    rs = avg_gain/avg_loss

    rsi = 100 - (100/ (1+rs))
    
    return rsi

def calculate_rsi_ewma(data, column="Adj Close", window=14):

    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input data must be a pandas DataFrame")

    if column not in data.columns:
        raise ValueError(f"Column '{column}' not found in the DataFrame")

    # Calculate daily price changes (delta)
    delta = data[column].diff()

    # Separate gains and losses
    gains = delta.clip(lower=0)
    losses = -delta.clip(upper=0)
    
    avg_gain = gains.ewm(span=window, adjust=False).mean()
    avg_loss = losses.ewm(span=window, adjust=False).mean()

    rs = avg_gain/avg_loss

    rsi = 100 - (100/ (1+rs))
    
    return rsi

In [3]:
column = "Adj Close"

data['RSI'] = calculate_rsi(data=data, column=column, window=30)

data['RSI_ewma'] = calculate_rsi_ewma(data=data, column=column, window=14)

data

NameError: name 'data' is not defined

In [24]:
data

Price,Adj Close,Close,High,Low,Open,Volume,RSI,RSI_ewma
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL,,
Date,,,,,,,,
2020-01-02,72.796005,75.087502,75.150002,73.797501,74.059998,135480400,NaN,NaN
2020-01-03,72.088303,74.357498,75.144997,74.125000,74.287498,146322800,NaN,0.000000
2020-01-06,72.662720,74.949997,74.989998,73.187500,73.447502,118387200,NaN,11.100954
2020-01-07,72.320953,74.597504,75.224998,74.370003,74.959999,108872000,NaN,10.314862
2020-01-08,73.484322,75.797501,76.110001,74.290001,74.290001,132079200,NaN,29.831016
...,...,...,...,...,...,...,...,...
2023-12-05,192.477051,193.419998,194.399994,190.179993,190.210007,66628400,NaN,70.647406
2023-12-06,191.382416,192.320007,194.759995,192.110001,194.449997,41089700,NaN,63.738579
